In [1]:
#@title pip installs
!pip install pandas

     |████████████████████████████████| 11.3 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 23.0 MB/s eta 0:00:01
You should consider upgrading via the '/Users/andrea/Documents/git/access_layer/.venv/bin/python3 -m pip install --upgrade pip' command.


In [41]:
#@title Imports and setting up Access Layer Calls
import os
import requests

import pandas as pd

import pprint
pp = pprint.PrettyPrinter(indent=4)

_SERVICE_URL = 'https://development-admin-app-backend-2jv3ndpkoa-uc.a.run.app/'
headers = {
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
}

email = "aiteam@functionhealth.com"
password= "password"  # @param {type:"string"}
query = """mutation {
  createToken(email: "%s", password: "%s") {
    accessToken
    tokenType
    __typename
  }
}
""" % (email, password)

access_layer_url = os.path.join(_SERVICE_URL, 'graphql')
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})
jwt_token = response.json().get("data", {}).get('createToken', {}).get('accessToken')
headers['Authorization'] = f'Bearer {jwt_token}'


## Example
Below is an example Access layer GraphQL Query for paginated biomarker results that have two variables added into the example --`skip` and `limit`.

Additional example queries can be found within this [directory](https://github.com/Function-Health/admin-app-fe-next/tree/dev/src/apollo/queries) but note that these examples are based on Typescript and not Python.

Note that when changing the below queries the return JSON will also change. For example if you were to query the `symptoms` tables the return JSON strcture will look something like `{"data": {"biomarkerResultsPaginated": {...})`

A full list of Access Layer Schemas can be found [here](https://github.com/Function-Health/admin-app-backend/tree/dev/app/schemas)

In [48]:
#@title Pull all patients from the Access Layer
skip = 0  # @param {type:"string"}
limit = 1000000  # @param {type:"string"}

query = """
{
  patients(skip: %s, limit: %s) {
  id
        biologicalSex
        dob
        ethnicOrigin
        motivational{
          tenYearHealthGoals
        }
        femaleReproductiveHealth{
          currentMenstrualStatus
          currentlyPregnant
          birthControlType
        }
        patientBehaviorInfo{
          diet
          smokingFrequency
          alcoholFrequency
          physicallyActive
          smoker
          alcohol
        }
        medicalHx{
          currentMedications
          surgeries
          hospitalizations
          chronicConditions
          chronicConditionDiagnosisDates
        }
      }
    }""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("patients", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


{   'data': {   'patients': [   {   'biologicalSex': 'Female',
                                    'dob': '1994-01-01',
                                    'ethnicOrigin': ['English, Irish, Italian'],
                                    'femaleReproductiveHealth': None,
                                    'id': 'e9afbfc1-3e73-4f39-8735-1327bcdb8b43',
                                    'medicalHx': {   'chronicConditionDiagnosisDates': [   ''],
                                                     'chronicConditions': [],
                                                     'currentMedications': [   'Adderall, '
                                                                               '5mg, '
                                                                               'once '
                                                                               'daily, '
                                                                               'ADD'],
                               

In [50]:
import pandas as pd
import json

json_struct = json.loads(access_layer_df.to_json(orient="records"))    
df_flat = pd.io.json.json_normalize(json_struct)

AttributeError: module 'pandas.io.json' has no attribute 'json_normalize'

In [49]:
access_layer_df.head()

,id,biologicalSex,dob,ethnicOrigin,motivational,femaleReproductiveHealth,patientBehaviorInfo,medicalHx
0,e9afbfc1-3e73-4f39-8735-1327bcdb8b43,Female,1994-01-01,"[English, Irish, Italian]","{'tenYearHealthGoals': 'improve digestion, low...",None,"{'diet': ['Omnivore'], 'smokingFrequency': 'Me...","{'currentMedications': ['Adderall, 5mg, once d..."
1,26575945-ac8b-493c-a3a2-02ccaa98c6d0,Female,1990-02-07,[white],None,None,None,None
2,c962dcd8-db9e-4b84-8b70-38d5f4ff2ddf,Female,1987-01-01,[South Korean],{'tenYearHealthGoals': 'Figure out the root ca...,{'currentMenstrualStatus': 'Premenopause (befo...,"{'diet': ['Omnivore'], 'smokingFrequency': 'Wo...","{'currentMedications': ['Vitamin D, 2,000IU da..."
3,4e411606-07cc-4c93-8bdd-3335f0229b16,Male,1982-01-01,[Ashkenazi Jew],"{'tenYearHealthGoals': 'Lose fat, gain muscle,...","{'currentMenstrualStatus': None, 'currentlyPre...","{'diet': ['Omnivore'], 'smokingFrequency': Non...",{'currentMedications': ['Magnesium L threonate...
4,8f8979af-45cf-4049-9809-4b021d3941b4,Female,1980-05-07,[white],None,None,None,None


In [ ]:
#@title Pull single patient from the Access Layer
patient_id = "ceda71be-5706-43cb-a5b3-8ed4f6aa275f" # @param {type:"string"}

query = """
query patients($filterOps: PatientFilterOps) {
        patients(filterOps: $filterOps) {
            id
            biologicalSex
            dob
            fname
            preferredName
            ethnicOrigin
            patientContactInfo {
                email
            }
            motivational {
                tenYearHealthGoals
            }
            femaleReproductiveHealth {
                currentMenstrualStatus
                currentlyPregnant
                birthControlType
            }
            patientBehaviorInfo{
                diet
                smokingFrequency
                alcoholFrequency
                physicallyActive
                smoker
                alcohol
            }
            medicalHx {
                currentMedications
                surgeries
                hospitalizations
                chronicConditions
                chronicConditionDiagnosisDates
            }
        }
    }
"""
variables = {
  "filterOps": {
      "id": patient_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("patients", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull all Requisitions from the Access Layer
skip = 0  # @param {type:"string"}
limit = 10  # @param {type:"string"}
patient_id = "f4cdc35c-4608-4779-861d-1ec857801dd9" # @param {type:"string"}

query = """
query requisitions($filterOps: RequisitionFilterOps) {
        requisitions(filterOps: $filterOps) {
            id
            patientId
            allResultsAvailable
            reviewed
            dateTimeReviewed
            dateTimeConfirmed
            visits{
                id
                visitDate
                biomarkerResults {
                    questBiomarkerId
                    biomarkerName
                    testResult
                    measurementUnits
                    testResultOutOfRange
                    questReferenceRange
                }
            }
        }
    }
"""
variables = {
  "filterOps": {
      "patientId": patient_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("requisitions", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [ ]:
#@title Pull Recommendations by ID

recommendation_id = "6bc8603e-043e-4407-a188-3033f4300b41" # @param {type:"string"}

query = """
query recommendations($filterOps: RecommendationFilterOps) {
        recommendations(filterOps: $filterOps) {
            id
            outOfRangeType
            summaryDescription
            causesDescription
            symptomsDescription
            foodsToEatDescription
            foodsToAvoidDescription
            supplementsDescription
            selfCareDescription
            additionalTestsDescription
            followUpsDescription
        }
    }
"""
variables = {
  "filterOps": {
      "id": recommendation_id
  }
}

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={
        "query": query,
        "variables": variables
      })

access_layer_return = response.json()
pp.pprint(access_layer_return)

extract_access_layer_data = access_layer_return.get(
    "data", {}).get("recommendations", [])
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


In [21]:
#@title Pull all recommendations
skip = 0  # @param {type:"string"}
limit = 1000000 # @param {type:"string"}

query = """
{
  recommendations(skip: %s, limit: %s) {
    id
    name
    summaryDescription
    outOfRangeType
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
#pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("recommendations", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
print(access_layer_df)


                                       id  \
0    ec00f096-c263-4b25-9564-9eb48a371cae   
1    a46b1bf3-1435-4419-bb6b-80f9b4cfecd3   
2    c99cddf7-e3c5-4e7f-833f-af4ef58acd5c   
3    6bc8603e-043e-4407-a188-3033f4300b41   
4    3ce82f4e-caf6-4f64-9909-a4ea2b898d4f   
..                                    ...   
993  55aa3196-0bd8-4bd0-9627-0e8594d52268   
994  56423428-f96e-4d50-a437-7b3324816205   
995  b754495e-522c-4c9e-87e5-d8e06c373021   
996  6f6cb1ae-2b2f-4684-82fa-073dbf5f850a   
997  b1373745-cb60-4f61-b519-71e9dce5bec3   

                                                  name  \
0          PLATELET COUNT out of range - below for All   
1            MUGWORT (W6) IGE - CLASS in range for All   
2                          PSA, FREE in range for Male   
3      Celiac Immunoglobulin A, (IgA) in range for All   
4                                  Creatinine in range   
..                                                 ...   
993                        LDL Medium in range for Al

In [26]:
len(access_layer_df)

998

In [22]:
access_layer_df.name.nunique()

626

In [23]:
access_layer_df.head()

,id,name,summaryDescription,outOfRangeType
0,ec00f096-c263-4b25-9564-9eb48a371cae,PLATELET COUNT out of range - below for All,None,below
1,a46b1bf3-1435-4419-bb6b-80f9b4cfecd3,MUGWORT (W6) IGE - CLASS in range for All,Your results indicate you do not have an aller...,
2,c99cddf7-e3c5-4e7f-833f-af4ef58acd5c,"PSA, FREE in range for Male",Your healthy prostate levels go hand in hand w...,None
3,6bc8603e-043e-4407-a188-3033f4300b41,"Celiac Immunoglobulin A, (IgA) in range for All",A totally negative (zero antibodies) antibody ...,
4,3ce82f4e-caf6-4f64-9909-a4ea2b898d4f,Creatinine in range,Continue to support your healthy creatinine le...,None


In [33]:
access_layer_df.groupby('name').agg({'summaryDescription': pd.Series.nunique}).reset_index().sort_values('summaryDescription', ascending = False).head(10)

,name,summaryDescription
298,JOHNSON GRASS (G10) IGE in range for All,2
296,JOHNSON GRASS (G10) IGE - CLASS in range for All,2
566,TIMOTHY GRASS (G6) IGE in range for All,2
515,SEX HORMONE BINDING GLOBULIN in range,2
445,PERENNIAL RYE GRASS (G5) IGE - CLASS in range ...,2
447,PERENNIAL RYE GRASS (G5) IGE in range for All,2
293,Iron % Saturation out of range - below for All,2
555,"TESTOSTERONE, FREE in range",2
524,SHRIMP (F24) IGE in range for All,2
382,MUGWORT (W6) IGE - CLASS in range for All,2


In [32]:
list(access_layer_df[access_layer_df['name'] == 'JOHNSON GRASS (G10) IGE in range for All']['summaryDescription'])

['Your results indicate you do not have an allergy to any of the above-listed allergens.',
 'Your results indicate you do not have an allergy to any of the large classes of grass found in the United States.',
 'Your results indicate you do not have an allergy to any of the above-listed allergens.',
 'Your results indicate you do not have an allergy to any of the above-listed allergens.',
 'Your results indicate you do not have an allergy to any of the above-listed allergens.']

In [36]:
list(access_layer_df[access_layer_df['name'] == 'Iron % Saturation out of range - below for All']['summaryDescription'])

['Low levels suggest the likelihood of an iron deficiency. This can lead to iron-deficiency anemia, which causes the body to create smaller-than-normal red blood cells and less hemoglobin. You’ll want to know the cause to treat and correct this properly.',
 'About 10% of the population has high iron, but excessive iron usually needs to be managed. Your high levels could be a result of a couple of things you’ll want to look into before seeking a solution.']

In [37]:
list(access_layer_df[access_layer_df['name'] == 'SEX HORMONE BINDING GLOBULIN in range']['summaryDescription'])

['Your levels are great. Maintaining a healthy lifestyle through good foods and exercise can help maintain the balance here. A decrease in SHBG can lead to an increase in male sex hormones and vice versa.',
 'Your levels are great. Maintaining a healthy lifestyle through good foods and exercise can help maintain balance. SHBG is best when in balance. Pay attention if your menstrual cycle ever becomes irregular or if you experience hormonal acne outbreaks.']

In [38]:
access_layer_df.summaryDescription.nunique()

255

In [42]:
skip = 0  # @param {type:"string"}
limit = 1000  # @param {type:"string"}

# An example Access layer GraphQL Query can be found below with two variables
# Set.
query = """
{
  biomarkerResultsPaginated(skip: %s, limit: %s) {
    items {
      id
      biomarkerName
    }
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("biomarkerResultsPaginated", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
print(access_layer_df)


                                                 items
0    {'id': '78079c00-f17a-4304-94ca-78a1d93f95b9',...
1    {'id': '032e4b34-9040-4493-9671-ee0e3e891f99',...
2    {'id': '7b540cbf-261b-46b1-af1a-ed3d0de5adcf',...
3    {'id': '36d371ef-3815-4fca-8fee-3b446aa373dc',...
4    {'id': 'c3d8922a-5d3e-4349-8c2f-f92b2ee65b4c',...
..                                                 ...
995  {'id': 'efa6a0e3-10d4-44d3-8edf-9c1ed60e1491',...
996  {'id': '58573644-d7ff-49c6-b2c4-c30b83ef6c21',...
997  {'id': '59b85360-6b54-4703-a868-59149be95e48',...
998  {'id': '69f1bdd2-849b-4751-af85-6c30c0348b21',...
999  {'id': 'd72f6f29-b287-4a33-8d7c-15c2ac24ce27',...

[1000 rows x 1 columns]


In [43]:
access_layer_df.head()

,items
0,"{'id': '78079c00-f17a-4304-94ca-78a1d93f95b9',..."
1,"{'id': '032e4b34-9040-4493-9671-ee0e3e891f99',..."
2,"{'id': '7b540cbf-261b-46b1-af1a-ed3d0de5adcf',..."
3,"{'id': '36d371ef-3815-4fca-8fee-3b446aa373dc',..."
4,"{'id': 'c3d8922a-5d3e-4349-8c2f-f92b2ee65b4c',..."


In [44]:
recs = access_layer_df['items']
biomarker_results = pd.json_normalize(access_layer_df['items'], meta = 'biomarkerName')
print(biomarker_results)

                                       id                        biomarkerName
0    78079c00-f17a-4304-94ca-78a1d93f95b9  Antinuclear Antibodies (ANA) Screen
1    032e4b34-9040-4493-9671-ee0e3e891f99                               Leptin
2    7b540cbf-261b-46b1-af1a-ed3d0de5adcf               Rheumatoid Factor (RF)
3    36d371ef-3815-4fca-8fee-3b446aa373dc          Omega 3 (EPA and DHA) Index
4    c3d8922a-5d3e-4349-8c2f-f92b2ee65b4c             Methylmalonic Acid (MMA)
..                                    ...                                  ...
995  efa6a0e3-10d4-44d3-8edf-9c1ed60e1491                                  RBC
996  58573644-d7ff-49c6-b2c4-c30b83ef6c21            SQUAMOUS EPITHELIAL CELLS
997  59b85360-6b54-4703-a868-59149be95e48                       MAGNESIUM, RBC
998  69f1bdd2-849b-4751-af85-6c30c0348b21                            LDL SMALL
999  d72f6f29-b287-4a33-8d7c-15c2ac24ce27                            POTASSIUM

[1000 rows x 2 columns]


In [46]:
list(biomarker_results['biomarkerName'].unique())

['Antinuclear Antibodies (ANA) Screen',
 'Leptin',
 'Rheumatoid Factor (RF)',
 'Omega 3 (EPA and DHA) Index',
 'Methylmalonic Acid (MMA)',
 'Omega-3 Index Risk',
 'Thyroglobulin Antibodies (TgAb)',
 'Omega 6 / Omega 3 Ratio',
 'Omega 3 and 6: Arachidonic Acid/EPA Ratio ',
 'Follicle Stimulating Hormone (FSH)',
 'Omega 6: Arachidonic Acid',
 'Thyroid Peroxidase Antibodies (TPO)',
 'Omega 3: EPA',
 'Luteinizing Hormone (LH)',
 'Homocysteine',
 'DHEA Sulfate',
 'Prolactin',
 'Omega 3: DHA',
 'Testosterone, Free',
 'Prostate Specific Antigen (PSA), Free',
 'Estradiol (E2)',
 'Sex Hormone Binding Globulin (SHBG)',
 'Prostate Specific Antigen (PSA), Total',
 'Prostate Specific Antigen (PSA) %, Free',
 'Amylase',
 'Lipase',
 'Zinc',
 'Lead ',
 'Testosterone, Total',
 'Potassium',
 'Iron',
 'Iron Binding Capacity',
 'Iron % Saturation',
 'Ferritin',
 'Uric Acid',
 'Globulin',
 'Glucose',
 'Blood Urea Nitrogen (BUN)',
 'Chloride',
 'Creatinine',
 'Albumin / Globulin Ratio',
 'Total Bilirubin',


In [39]:
#@title Pull all Biomarker Results from the Access Layer
skip = 0  # @param {type:"string"}
limit = 1000000 # @param {type:"string"}

query = """
{
  biomarkerResults(skip: %s, limit: %s) {
    biomarkerName
    questReferenceRange
  }
}
""" % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("biomarkerResults", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
pp.pprint(access_layer_df)


{   'data': None,
    'errors': [   {   'locations': [{'column': 3, 'line': 3}],
                      'message': 'Not authenticated',
                      'path': ['biomarkerResults']}]}


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
#@title Pull all recommendations
skip = 0  # @param {type:"string"}
limit = 1000000 # @param {type:"string"}

query = """
    {
      patientById(id: "%s"){
        id
        biologicalSex
        dob
        ethnicOrigin
        motivational{
          tenYearHealthGoals
        }
        femaleReproductiveHealth{
          currentMenstrualStatus
          currentlyPregnant
          birthControlType
        }
        patientBehaviorInfo{
          diet
          smokingFrequency
          alcoholFrequency
          physicallyActive
          smoker
          alcohol
        }
        medicalHx{
          currentMedications
          surgeries
          hospitalizations
          chronicConditions
          chronicConditionDiagnosisDates
        }
      }
    }
    """ % (skip, limit)

# Call Access Layer
response = requests.post(
    access_layer_url,
    headers=headers,
    json={'query': query})

access_layer_return = response.json()
#pp.pprint(access_layer_return)
extract_access_layer_data = access_layer_return.get(
    "data", {}).get("recommendations", {})
access_layer_df = pd.DataFrame(extract_access_layer_data)

# Print the resulting dataframe
print(access_layer_df)

In [ ]:
query = """
    {
      patientById(id: "%s"){
        id
        biologicalSex
        dob
        ethnicOrigin
        motivational{
          tenYearHealthGoals
        }
        femaleReproductiveHealth{
          currentMenstrualStatus
          currentlyPregnant
          birthControlType
        }
        patientBehaviorInfo{
          diet
          smokingFrequency
          alcoholFrequency
          physicallyActive
          smoker
          alcohol
        }
        medicalHx{
          currentMedications
          surgeries
          hospitalizations
          chronicConditions
          chronicConditionDiagnosisDates
        }
      }
    }
    """